# Fusion Meta-Model (TCS 2021–2023, Sentiment-Gated)

**Purpose.** Build and evaluate the single-symbol (TCS) fusion expert that stacks OHLCV, sentiment (lag-1 gated), and pattern models for the 2021–2023 horizon. The notebook recreates the same preprocessing used in production so that the meta learner can be re-run or audited without touching the larger orchestration code.

**What this covers.**
- Restores repo + DVC state, including the metadata that defines split boundaries and label semantics.
- Loads the latest registered base experts from MLflow and generates leakage-safe OOF probabilities for the train period, plus gated predictions for validation/test.
- Performs a grid search over logistic-regression meta parameters, selects via validation ROC-AUC, and evaluates the champion on both validation and holdout data.
- Logs lineage artifacts, ROC metrics, and the frozen fusion model to `dissertation-fusion-prediction`, then registers the run for downstream consumption.

**Artifacts emitted.** `meta_val_candidate_ranking.csv`, MLflow tags linking git+DVC hashes, and a registered model named `fusion_meta_lr_tcs_2021_2023_sent_lag_1_gated`.

Run the cells sequentially (Colab friendly). If any dependency step fails, rerun from that cell to ensure consistent lineage.

## 0) Prerequisites: Pip Installs

In [1]:
# Install deps (Colab)
!pip -q install feast==0.58.0 s3fs boto3 pyarrow pandas scikit-learn "mlflow>=2.12,<3" "dagshub==0.6.4" "dvc[s3]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44

## 1) Credentials / Environment
Set these using Colab `userdata` (or replace with your own method).

In [2]:
import os
from google.colab import userdata

# AWS
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = userdata.get('AWS_DEFAULT_REGION') or 'us-east-1'

# MLflow (DAGsHub)
os.environ["DAGSHUB_USER_TOKEN"] = userdata.get('DAGSHUB_TOKEN')

missing = [k for k in ['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY','DAGSHUB_USER_TOKEN'] if not os.environ.get(k)]
if missing:
    raise ValueError(f"Missing secrets in Colab userdata: {missing}")
print('Secrets loaded:', ', '.join(['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY','AWS_DEFAULT_REGION','DAGSHUB_USER_TOKEN']))

Secrets loaded: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_DEFAULT_REGION, DAGSHUB_USER_TOKEN


## 2) Setup + DVC pull

In [3]:
import json
from pathlib import Path
import pandas as pd
import numpy as np

REPO_NAME = "dissertation"  # change if needed
REPO_URL = f"https://github.com/Roncool13/{REPO_NAME}.git"
BRANCH_OR_COMMIT = "main"  # or a specific commit hash for perfect reproducibility

if not Path(REPO_NAME).exists():
    !git clone {REPO_URL}
%cd {REPO_NAME}
!git checkout {BRANCH_OR_COMMIT}
!git status

!dvc pull -q
print("DVC pull done ✅")

Cloning into 'dissertation'...
remote: Enumerating objects: 815, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (206/206), done.
Receiving objects: 100% (815/815), 275.04 KiB | 7.86 MiB/s, done.
remote: Total 815 (delta 196), reused 262 (delta 118), pack-reused 470 (from 1)
Resolving deltas: 100% (493/493), done.
/content/dissertation
Already on 'main'
Your branch is up to date with 'origin/main'.
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
DVC pull done ✅


## 3) MLflow Setup

In [4]:
import mlflow
import dagshub

dagshub.init(repo_owner='Roncool13', repo_name='dissertation-mlflow', mlflow=True)
mlflow.set_experiment("dissertation-fusion-prediction")
mlflow.sklearn.autolog(disable=True)

print('MLflow tracking URI:', mlflow.get_tracking_uri())

Accessing as Roncool13

Initialized MLflow to track repo "Roncool13/dissertation-mlflow"

Repository Roncool13/dissertation-mlflow initialized!

MLflow tracking URI: https://dagshub.com/Roncool13/dissertation-mlflow.mlflow


## 4) Load datasets + metadata + label name

In [80]:
meta_ohlcv = json.loads(Path(f"data/features/ohlcv_feature_metadata.json").read_text())
horizon_days = int(meta_ohlcv.get("horizon_days", 5))
label_col = f"y_up_{horizon_days}d"

ohlcv = pd.read_parquet("data/features/ohlcv_features.parquet")
sent  = pd.read_parquet("data/features/news_sentiment_features.parquet")
pat   = pd.read_parquet("data/features/pattern_features.parquet")
news_meta = json.loads(Path("data/features/news_sentiment_feature_metadata.json").read_text())

for df in (ohlcv, sent, pat):
    df["date"] = pd.to_datetime(df["date"])

assert label_col in ohlcv.columns, f"{label_col} not found in ohlcv_features"
print("Using label:", label_col)

Using label: y_up_5d


## 5) Build X columns for each expert

### 5.1) OHLCV X columns

In [81]:
# Exclude forward-looking columns to prevent leakage (matches your OHLCV notebook idea)
exclude_prefixes = ("close_fwd_", "ret_fwd_")
exclude_cols = {"symbol", "date", label_col}

ohlcv_X_cols = [
    c for c in ohlcv.columns
    if c not in exclude_cols and not c.startswith(exclude_prefixes)
]
print("OHLCV feature count:", len(ohlcv_X_cols))

OHLCV feature count: 85


### 5.2) Sentiment X columns (Building Gated Sentiment for 1 day lag)

In [82]:
label_col = news_meta["supervision"]["label_col_name"]
sent_X_cols = [c for c in sent.columns if c not in {"symbol","date", label_col}]
print("OHLCV feature count:", len(sent_X_cols))

OHLCV feature count: 127


In [83]:
# All trading days (ground truth timeline)
calendar = (
    ohlcv[["symbol", "date"]]
    .drop_duplicates()
    .sort_values(["symbol", "date"])
    .reset_index(drop=True)
)

In [84]:
sent_full = calendar.merge(
    sent,
    on=["symbol", "date"],
    how="left",
    suffixes=("", "_sent")
)

In [85]:
# Binary gate
sent_full["news_present"] = sent_full["news_present"].fillna(0.0)

# Counts & intensities → 0 when no news
zero_cols = [
    "article_count", "pos_count", "neg_count", "neu_count",
    "highrel_count", "sent_cov", "sent_abs",
    "highrel_impact", "imbalance"
]

ratio_cols = [
    "pos_ratio", "neg_ratio", "neu_ratio", "highrel_ratio"
]

polarity_cols = [
    "polarity_mean", "polarity_std",
    "polarity_wmean",
    "highrel_polarity_mean",
    "highrel_polarity_wmean"
]

for c in zero_cols + ratio_cols + polarity_cols:
    if c in sent_full.columns:
        sent_full[c] = sent_full[c].fillna(0.0)

In [87]:
sent = sent_full.sort_values(["symbol", "date"])

base_cols = [
    "article_count", "pos_ratio", "neg_ratio", "neu_ratio",
    "highrel_ratio", "polarity_wmean", "sent_cov",
    "imbalance", "news_present"
]

for lag in range(1, 6):
    for col in base_cols:
        sent[f"{col}_lag_{lag}"] = (
            sent.groupby("symbol")[col].shift(lag)
        )

### 5.3) Pattern X columns (+ shift by 1 day)

In [88]:
pat_exclude = {"symbol", "date"}
pat_X_cols = [c for c in pat.columns if c not in pat_exclude]

pat = pat.sort_values(["symbol","date"]).reset_index(drop=True)
pat[pat_X_cols] = pat.groupby("symbol", sort=False)[pat_X_cols].shift(1)

print("Pattern feature count:", len(pat_X_cols))

Pattern feature count: 17


In [44]:
# ---------- Build df with a SINGLE label source (avoid _x/_y collisions) ----------
if label_col in sent.columns and label_col in ohlcv.columns:
    print(f"Label Columns {label_col} found in both sentiment and ohlcv dataframes. Removing it from sentiment to avoid clash")
    # Drop label col from sentiment
    sent.drop(columns=[label_col], inplace=True)
    # Re-define sent_X_cols after dropping the label column
    sent_X_cols = [c for c in sent.columns if c not in sent_exclude]

assert label_col in ohlcv.columns, f"{label_col} not found in ohlcv_features"
assert label_col not in sent.columns, f"{label_col} found in sentiment_features even after dropping"
print(f"Sentiment Columns: {sent.columns.tolist()}")

Label Columns y_up_5d found in both sentiment and ohlcv dataframes. Removing it from sentiment to avoid clash
Sentiment Columns: ['symbol', 'date', 'article_count', 'pos_count', 'neg_count', 'neu_count', 'highrel_count', 'rel_score_mean', 'sent_conf_mean', 'polarity_mean', 'polarity_std', 'polarity_wmean', 'highrel_polarity_mean', 'highrel_polarity_wmean', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'highrel_ratio', 'sent_cov', 'sent_abs', 'imbalance', 'highrel_impact', 'news_present', 'article_count_lag_1', 'pos_ratio_lag_1', 'neg_ratio_lag_1', 'neu_ratio_lag_1', 'highrel_ratio_lag_1', 'rel_score_mean_lag_1', 'sent_conf_mean_lag_1', 'polarity_mean_lag_1', 'polarity_std_lag_1', 'polarity_wmean_lag_1', 'highrel_polarity_mean_lag_1', 'highrel_polarity_wmean_lag_1', 'sent_cov_lag_1', 'sent_abs_lag_1', 'imbalance_lag_1', 'highrel_impact_lag_1', 'news_present_lag_1', 'article_count_lag_2', 'pos_ratio_lag_2', 'neg_ratio_lag_2', 'neu_ratio_lag_2', 'highrel_ratio_lag_2', 'rel_score_mean_lag_2', 'se

## 6) Merge all on (symbol, date) + create splits from OHLCV metadata

In [29]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [89]:
# Keep label and key from OHLCV
label_df = ohlcv[["symbol","date",label_col]].dropna().copy()

# Merge expert feature tables (after their shifts)
df = label_df.merge(
    ohlcv[["symbol","date"] + ohlcv_X_cols], on=["symbol","date"], how="inner"
).merge(
    sent[["symbol","date"] + sent_X_cols], on=["symbol","date"], how="inner"
).merge(
    pat[["symbol","date"] + pat_X_cols], on=["symbol","date"], how="inner"
)

df = df.sort_values(["symbol","date"]).reset_index(drop=True)
df = df.dropna().reset_index(drop=True)

print("Merged rows:", len(df))
print("Date min/max:", df["date"].min(), df["date"].max())

Merged rows: 598
Date min/max: 2021-02-25 00:00:00 2023-09-07 00:00:00


In [92]:
sent.columns.tolist()

['symbol',
 'date',
 'article_count',
 'pos_count',
 'neg_count',
 'neu_count',
 'highrel_count',
 'rel_score_mean',
 'sent_conf_mean',
 'polarity_mean',
 'polarity_std',
 'polarity_wmean',
 'highrel_polarity_mean',
 'highrel_polarity_wmean',
 'pos_ratio',
 'neg_ratio',
 'neu_ratio',
 'highrel_ratio',
 'sent_cov',
 'sent_abs',
 'imbalance',
 'highrel_impact',
 'news_present',
 'article_count_lag_1',
 'pos_ratio_lag_1',
 'neg_ratio_lag_1',
 'neu_ratio_lag_1',
 'highrel_ratio_lag_1',
 'rel_score_mean_lag_1',
 'sent_conf_mean_lag_1',
 'polarity_mean_lag_1',
 'polarity_std_lag_1',
 'polarity_wmean_lag_1',
 'highrel_polarity_mean_lag_1',
 'highrel_polarity_wmean_lag_1',
 'sent_cov_lag_1',
 'sent_abs_lag_1',
 'imbalance_lag_1',
 'highrel_impact_lag_1',
 'news_present_lag_1',
 'article_count_lag_2',
 'pos_ratio_lag_2',
 'neg_ratio_lag_2',
 'neu_ratio_lag_2',
 'highrel_ratio_lag_2',
 'rel_score_mean_lag_2',
 'sent_conf_mean_lag_2',
 'polarity_mean_lag_2',
 'polarity_std_lag_2',
 'polarity_wmea

In [93]:
def parse_date(s):
    return pd.to_datetime(s) if s else None

splits = meta_ohlcv.get("splits", {})
train_start = parse_date(splits.get("train_start"))
train_end   = parse_date(splits.get("train_end"))
val_start   = parse_date(splits.get("val_start"))
val_end     = parse_date(splits.get("val_end"))
test_start  = parse_date(splits.get("test_start"))
test_end    = parse_date(splits.get("test_end"))

if all([train_start, train_end, val_start, val_end, test_start]):
    train_df = df[(df["date"]>=train_start) & (df["date"]<=train_end)].copy()
    val_df   = df[(df["date"]>=val_start) & (df["date"]<=val_end)].copy()
    test_df  = df[(df["date"]>=test_start) & ((df["date"]<=test_end) if test_end is not None else True)].copy()
else:
    # fallback: 2021=train, 2022=val, 2023=test style
    yrs = df["date"].dt.year
    train_df = df[yrs==2021].copy()
    val_df   = df[yrs==2022].copy()
    test_df  = df[yrs==2023].copy()

print("train rows", len(train_df), train_df["date"].min(), train_df["date"].max(), "pos_rate", float(train_df[label_col].mean()))
print("val rows", len(val_df), val_df["date"].min(), val_df["date"].max(), "pos_rate", float(val_df[label_col].mean()))
print("test rows", len(test_df), test_df["date"].min(), test_df["date"].max(), "pos_rate", float(test_df[label_col].mean()))

train rows 205 2021-02-25 00:00:00 2021-12-31 00:00:00 pos_rate 0.6097560975609756
val rows 231 2022-01-03 00:00:00 2022-12-30 00:00:00 pos_rate 0.49783549783549785
test rows 162 2023-01-02 00:00:00 2023-09-07 00:00:00 pos_rate 0.5493827160493827


In [94]:
# Sanity checks: alignment across modalities + leakage guardrails
import numpy as np
import pandas as pd

# 1) Keys must be unique and sorted
assert df[['symbol','date']].isna().sum().sum() == 0, "Nulls in (symbol,date)"
assert not df.duplicated(['symbol','date']).any(), "Duplicate (symbol,date) rows found"
df = df.sort_values(['symbol','date']).reset_index(drop=True)

# 2) Feature sets exist and are non-empty
for name, cols in [('OHLCV', ohlcv_X_cols), ('SENT', sent_X_cols), ('PAT', pat_X_cols)]:
    missing = [c for c in cols if c not in df.columns]
    assert len(missing) == 0, f"{name} missing columns: {missing[:10]} (showing first 10)"
    assert len(cols) > 0, f"{name} has 0 features"

# 3) Split frames must be perfectly aligned by construction (they come from df masks),
# but we still verify shapes and key hashes so any accidental reordering is caught.
def _key_hash(frame: pd.DataFrame) -> str:
    # stable hash of (symbol,date) order
    return pd.util.hash_pandas_object(frame[['symbol','date']], index=False).astype('uint64').sum().__str__()

for name, part in [('train', train_df), ('val', val_df), ('test', test_df)]:
    assert part[['symbol','date']].isna().sum().sum() == 0
    assert not part.duplicated(['symbol','date']).any()
    print(f"{name}: rows={len(part)} key_hash={_key_hash(part)} "
          f"pos_rate={float(part[label_col].mean()):.4f} "
          f"date_min={part['date'].min().date()} date_max={part['date'].max().date()}")

# 4) Leakage sanity: features are shifted already, but we still ensure no forward-looking columns sneak in
leaky = [c for c in df.columns if c.startswith(('close_fwd_', 'ret_fwd_'))]
assert len(leaky) == 0, f"Forward-looking columns present in df: {leaky[:10]}"


train: rows=205 key_hash=15295034474200279777 pos_rate=0.6098 date_min=2021-02-25 date_max=2021-12-31
val: rows=231 key_hash=9090236632237331621 pos_rate=0.4978 date_min=2022-01-03 date_max=2022-12-30
test: rows=162 key_hash=10320201828196904199 pos_rate=0.5494 date_min=2023-01-02 date_max=2023-09-07


## 7) Load expert models from MLflow Registry

In [95]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

client = MlflowClient()

def load_model_version(model_name: str, version: int = None):
    if version is None:
        # If no specific version is requested, find the latest
        versions = client.search_model_versions(f"name='{model_name}'")
        if not versions:
            raise ValueError(f"No registered model versions found for: {model_name}")
        v = max(int(m.version) for m in versions)
    else:
        # Use the specified version
        v = version

    uri = f"models:/{model_name}/{v}"
    print(f"Loading {model_name} v{v} -> {uri}")

    # Prefer native sklearn estimator so we can call predict_proba
    try:
        model = mlflow.sklearn.load_model(uri)
        model_kind = "sklearn"
    except Exception as e:
        print(f"mlflow.sklearn.load_model failed ({e}); falling back to pyfunc")
        model = mlflow.pyfunc.load_model(uri)
        model_kind = "pyfunc"

    return model, model_kind, uri

m_ohlcv, kind_ohlcv, uri_ohlcv = load_model_version("ohlcv_lr_baseline_tcs_2021_2023")
m_sent,  kind_sent,  uri_sent  = load_model_version("sentiment_lr_baseline_tcs_2021_2023", 7)
m_pat,   kind_pat,   uri_pat   = load_model_version("pattern_lr_baseline_tcs_2021_2023")  # change name if different


Loading ohlcv_lr_baseline_tcs_2021_2023 v2 -> models:/ohlcv_lr_baseline_tcs_2021_2023/2


Loading sentiment_lr_baseline_tcs_2021_2023 v7 -> models:/sentiment_lr_baseline_tcs_2021_2023/7


Loading pattern_lr_baseline_tcs_2021_2023 v1 -> models:/pattern_lr_baseline_tcs_2021_2023/1


## 8) Generate expert probabilities (meta features)

In [96]:
# OOF helper for time-series stacking (expanding-window CV)
import numpy as np
from sklearn.base import clone
from sklearn.model_selection import TimeSeriesSplit

def _ensure_series(y):
    # Accept pandas Series or array-like
    try:
        import pandas as pd
        if isinstance(y, (pd.Series, pd.DataFrame)):
            return y.squeeze()
    except Exception:
        pass
    return y

def oof_proba_timeseries(estimator, X, y, n_splits=5):
    """Generate out-of-fold probabilities for a time-ordered dataset.

    Notes:
    - Uses TimeSeriesSplit so each fold trains on past data and predicts a future block.
    - Earliest rows will not receive OOF predictions (by design). We return a valid_mask.
    """
    y = _ensure_series(y)

    # Basic alignment sanity checks (when pandas is used)
    try:
        import pandas as pd
        if isinstance(X, (pd.DataFrame, pd.Series)) and isinstance(y, pd.Series):
            if not X.index.equals(y.index):
                raise ValueError("Index mismatch: X and y are not aligned. Ensure all modalities were merged on identical keys and sorted identically.")
    except Exception:
        pass

    n = len(y)
    oof = np.full(n, np.nan, dtype=float)

    splitter = TimeSeriesSplit(n_splits=n_splits)

    # If X is numpy, we still can index with arrays
    for fold, (tr, te) in enumerate(splitter.split(np.arange(n))):
        # Skip folds where train labels are single-class (predict_proba may fail)
        y_tr = y.iloc[tr] if hasattr(y, "iloc") else y[tr]
        uniq = np.unique(y_tr)
        if len(uniq) < 2:
            continue

        m = clone(estimator)
        X_tr = X.iloc[tr] if hasattr(X, "iloc") else X[tr]
        X_te = X.iloc[te] if hasattr(X, "iloc") else X[te]
        m.fit(X_tr, y_tr)
        oof[te] = m.predict_proba(X_te)[:, 1]

    valid_mask = ~np.isnan(oof)
    return oof, valid_mask


In [104]:
# Generate expert probabilities (meta features) - OOF-safe for TRAIN
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.model_selection import TimeSeriesSplit

# --- Sanity: perfect row alignment across modalities ---
def _assert_aligned(df_base, cols, name):
    if len(df_base) == 0:
        raise ValueError(f"{name}: empty dataframe.")
    if df_base[cols].isna().any().any():
        # Allow NaNs only if you intentionally model missingness; here we want clean matrices.
        nan_cols = df_base[cols].columns[df_base[cols].isna().any()].tolist()
        raise ValueError(f"{name}: NaNs found in feature columns: {nan_cols[:10]} (showing up to 10).")

# Prepare matrices (all share the same index/order because they come from train_df/val_df/test_df)
X_ohlcv_train = train_df[ohlcv_X_cols].copy()
X_sent_train  = train_df[sent_X_cols].copy()
X_pat_train   = train_df[pat_X_cols].copy()
y_train       = train_df[label_col].astype(int).copy()

X_ohlcv_val = val_df[ohlcv_X_cols].copy()
X_sent_val  = val_df[sent_X_cols].copy()
X_pat_val   = val_df[pat_X_cols].copy()
y_val       = val_df[label_col].astype(int).copy()

X_ohlcv_test = test_df[ohlcv_X_cols].copy()
X_sent_test  = test_df[sent_X_cols].copy()
X_pat_test   = test_df[pat_X_cols].copy()
y_test       = test_df[label_col].astype(int).copy()

_assert_aligned(train_df, ohlcv_X_cols, "OHLCV/train")
_assert_aligned(train_df, sent_X_cols,   "Sentiment/train")
_assert_aligned(train_df, pat_X_cols,    "Pattern/train")

# (A) OOF predictions for TRAIN (meta-train) using expanding-window CV
N_SPLITS_OOF = 5
p_ohlcv_train_oof, m1 = oof_proba_timeseries(m_ohlcv, X_ohlcv_train, y_train, n_splits=N_SPLITS_OOF)
p_sent_train_oof,  m2 = oof_proba_timeseries(m_sent,  X_sent_train,  y_train, n_splits=N_SPLITS_OOF)
p_pat_train_oof,   m3 = oof_proba_timeseries(m_pat,   X_pat_train,   y_train, n_splits=N_SPLITS_OOF)

valid_oof = (m1 & m2 & m3)
dropped = int((~valid_oof).sum())
print(f"OOF coverage: {valid_oof.mean():.3f} (dropping {dropped} earliest train rows with no OOF coverage)")

meta_train = pd.DataFrame(
    {
        "p_ohlcv": p_ohlcv_train_oof,
        "p_sent":  p_sent_train_oof,
        "p_pat":   p_pat_train_oof,
    },
    index=train_df.index,
).loc[valid_oof].reset_index(drop=True)

# (B) Fit each expert on FULL TRAIN, then predict VAL/TEST (this is safe: no val/test leakage)
m_ohlcv_fit = clone(m_ohlcv).fit(X_ohlcv_train, y_train)
m_sent_fit  = clone(m_sent).fit(X_sent_train,  y_train)
m_pat_fit   = clone(m_pat).fit(X_pat_train,   y_train)

meta_val = pd.DataFrame(
    {
        "p_ohlcv": m_ohlcv_fit.predict_proba(X_ohlcv_val)[:, 1],
        "p_sent":  m_sent_fit.predict_proba(X_sent_val)[:, 1],
        "p_pat":   m_pat_fit.predict_proba(X_pat_val)[:, 1],
    }
)

meta_test = pd.DataFrame(
    {
        "p_ohlcv": m_ohlcv_fit.predict_proba(X_ohlcv_test)[:, 1],
        "p_sent":  m_sent_fit.predict_proba(X_sent_test)[:, 1],
        "p_pat":   m_pat_fit.predict_proba(X_pat_test)[:, 1],
    }
)

meta_train["p_sent"] = 1 - meta_train["p_sent"]
meta_val["p_sent"]   = 1 - meta_val["p_sent"]
meta_test["p_sent"]  = 1 - meta_test["p_sent"]

y_meta_train = y_train.loc[valid_oof].reset_index(drop=True)
y_meta_val = y_val
y_meta_test = y_test

# ----- Sentiment gating using lagged news presence -----
# IMPORTANT: Use the lagged flag (news_present_lag_1) so it's leakage-safe.

GATE_COL = "news_present_lag_1"   # must exist in your sentiment feature columns (sent_X_cols)

# Build gate vectors aligned to the SAME rows used in meta_* frames
gate_train = train_df.loc[valid_oof, GATE_COL].astype(float).reset_index(drop=True)
gate_val   = val_df[GATE_COL].astype(float).reset_index(drop=True)
gate_test  = test_df[GATE_COL].astype(float).reset_index(drop=True)

# gate_train = train_df.loc[valid_oof, GATE_COL].values
# gate_val   = val_df[GATE_COL].values
# gate_test  = test_df[GATE_COL].values

# Sanity: gate must be 0/1 (or close)
for name, g in [("train", gate_train), ("val", gate_val), ("test", gate_test)]:
    print(name, "gate value_counts:", g.value_counts(dropna=False).to_dict())

# Create gated sentiment probability
meta_train["p_sent_gated"] = meta_train["p_sent"] * gate_train
meta_val["p_sent_gated"]   = meta_val["p_sent"]   * gate_val
meta_test["p_sent_gated"]  = meta_test["p_sent"]  * gate_test

# Add news_present_lag_1 as separate feature
# meta_train[GATE_COL] = gate_train
# meta_val[GATE_COL]   = gate_val
# meta_test[GATE_COL]  = gate_test

# Optionally drop raw p_sent to force usage via gating only
meta_train = meta_train.drop(columns=["p_sent"])
meta_val   = meta_val.drop(columns=["p_sent"])
meta_test  = meta_test.drop(columns=["p_sent"])

print("meta_train/meta_val/meta_test shapes:", meta_train.shape, meta_val.shape, meta_test.shape)


OOF coverage: 0.829 (dropping 35 earliest train rows with no OOF coverage)
train gate value_counts: {1.0: 166, 0.0: 4}
val gate value_counts: {1.0: 216, 0.0: 15}
test gate value_counts: {1.0: 158, 0.0: 4}
meta_train/meta_val/meta_test shapes: (170, 3) (231, 3) (162, 3)


In [105]:
meta_train.head(2)

,p_ohlcv,p_pat,p_sent_gated
0,0.027064,0.482420,0.676924
1,0.008420,0.490947,0.548423


In [62]:
assert len(meta_train) == len(gate_train)
assert len(meta_val) == len(gate_val)
assert len(meta_test) == len(gate_test)

In [106]:
import numpy as np

def describe_probs(df, name):
    print("\n==", name, "==")
    for c in df.columns:
        arr = df[c].to_numpy()
        print(c,
              "min", float(np.min(arr)),
              "max", float(np.max(arr)),
              "mean", float(np.mean(arr)),
              "std", float(np.std(arr)),
              "unique~", len(np.unique(np.round(arr, 6))))

describe_probs(meta_train, "meta_train")
describe_probs(meta_val, "meta_val")
describe_probs(meta_test, "meta_test")


== meta_train ==
p_ohlcv min 1.60645624189986e-07 max 0.9984560985718312 mean 0.4242770599575667 std 0.3749822693635382 unique~ 162
p_pat min 0.479385531747014 max 0.5335696390757925 mean 0.5048939512426592 std 0.00957394612159749 unique~ 170
p_sent_gated min 0.0 max 0.8370500425631179 mean 0.4900867508695377 std 0.14941156717845927 unique~ 167

== meta_val ==
p_ohlcv min 0.13095935213198978 max 0.8037656381286176 mean 0.473666669790789 std 0.1286899205680463 unique~ 230
p_pat min 0.4751596376556519 max 0.5340669720453525 mean 0.4997305292606429 std 0.011546023003653623 unique~ 231
p_sent_gated min 0.0 max 0.9235052125484218 mean 0.4205900183055187 std 0.2074229469441116 unique~ 217

== meta_test ==
p_ohlcv min 0.2578894961584779 max 0.812039457309756 mean 0.591403629042295 std 0.11802803084366054 unique~ 162
p_pat min 0.4726631809902667 max 0.5439980699505257 mean 0.5018996098098693 std 0.010008041867135288 unique~ 161
p_sent_gated min 0.0 max 0.8767059092051304 mean 0.47241866786906

## 9) Train meta-model (LogReg baseline + TimeSeries CV on TRAIN, pick by VAL)

In [107]:
# ---- Meta-model training (OOF-safe meta features already built) ----
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef
)

SEED = 42

def eval_split(estimator, X, y, prefix: str):
    p = estimator.predict_proba(X)[:, 1]
    pred = (p >= 0.5).astype(int)
    return {
        f"{prefix}_accuracy": float(accuracy_score(y, pred)),
        f"{prefix}_roc_auc": float(roc_auc_score(y, p)) if len(np.unique(y)) > 1 else float("nan"),
        f"{prefix}_balanced_accuracy": float(balanced_accuracy_score(y, pred)),
        f"{prefix}_mcc": float(matthews_corrcoef(y, pred)) if len(np.unique(y)) > 1 else float("nan"),
    }

# Sanity: lengths + index alignment
assert len(meta_train) == len(y_meta_train), "meta_train and y_meta_train length mismatch"
assert len(meta_val)   == len(y_meta_val),   "meta_val and y_meta_val length mismatch"
assert len(meta_test)  == len(y_meta_test),  "meta_test and y_meta_test length mismatch"

print("Meta features columns:", list(meta_train.columns))
print("Meta TRAIN/VAL/TEST sizes:", len(meta_train), len(meta_val), len(meta_test))
print("Label pos-rate train/val/test:",
      float(np.mean(y_meta_train)),
      float(np.mean(y_meta_val)),
      float(np.mean(y_meta_test)))

# Grid for meta LR
C_grid = [1e-3, 1e-2, 1e-1, 1.0, 10.0]
pen_grid = ["l1", "l2"]
cw_grid = [None, "balanced"]

rows = []
cands = []
for C in C_grid:
    for pen in pen_grid:
        for cw in cw_grid:
            # liblinear supports l1/l2; saga also works but keep it simple/reproducible.
            clf = LogisticRegression(
                C=C, penalty=pen, class_weight=cw, solver="liblinear", max_iter=2000, random_state=SEED
            )
            model = Pipeline([("scaler", StandardScaler()), ("clf", clf)])
            model.fit(meta_train, y_meta_train)

            val_p = model.predict_proba(meta_val)[:, 1]
            val_auc = roc_auc_score(y_meta_val, val_p) if len(np.unique(y_meta_val)) > 1 else float("nan")

            rows.append({
                "C": C, "penalty": pen, "class_weight": str(cw),
                "val_roc_auc": float(val_auc),
            })
            cands.append(model)

rank = pd.DataFrame(rows).sort_values("val_roc_auc", ascending=False).reset_index(drop=True)
best_params = rank.iloc[0].to_dict()
best_idx = int(rank.index[0])

# Pick best and refit only on meta_train (do NOT use val for fitting)
final_meta = cands[rank.index[0]]

print("Best VAL params:", {k: best_params[k] for k in ["C","penalty","class_weight"]}, "VAL AUC:", best_params["val_roc_auc"])

val_metrics = eval_split(final_meta, meta_val, y_meta_val, "val")
test_metrics = eval_split(final_meta, meta_test, y_meta_test, "test")
print("VAL:", val_metrics)
print("TEST:", test_metrics)

# Keep these names expected by the MLflow cell below
X_train, y_train = meta_train, y_meta_train
X_val, y_val     = meta_val,   y_meta_val
X_test, y_test   = meta_test,  y_meta_test


Meta features columns: ['p_ohlcv', 'p_pat', 'p_sent_gated']
Meta TRAIN/VAL/TEST sizes: 170 231 162
Label pos-rate train/val/test: 0.5882352941176471 0.49783549783549785 0.5493827160493827
Best VAL params: {'C': 0.001, 'penalty': 'l1', 'class_weight': 'None'} VAL AUC: 0.5
VAL: {'val_accuracy': 0.49783549783549785, 'val_roc_auc': 0.5, 'val_balanced_accuracy': 0.5, 'val_mcc': 0.0}
TEST: {'test_accuracy': 0.5493827160493827, 'test_roc_auc': 0.5, 'test_balanced_accuracy': 0.5, 'test_mcc': 0.0}


In [110]:
print(final_meta.named_steps['clf'].coef_, final_meta.named_steps['clf'].intercept_)

[[0. 0. 0.]] [0.]


## 10) Visualizations (ROC-Curve)

In [111]:
# ---- MLflow logging + registration for meta-model ----
import mlflow
from mlflow.models.signature import infer_signature

mlflow.sklearn.autolog(disable=True)

run_name = f"stacked_meta_lr_tcs_{meta_ohlcv.get('start_year','2021')}_{meta_ohlcv.get('end_year','2023')}_h{horizon_days}d_sent_lag_1_gated"
with mlflow.start_run(run_name=run_name):
    mlflow.set_tag("stacking_inputs", "p_ohlcv,p_sent,p_pat")
    mlflow.set_tag("horizon_days", str(horizon_days))

    # Log chosen params
    mlflow.log_params({
        "meta_C": best_params["C"],
        "meta_penalty": best_params["penalty"],
        "meta_class_weight": best_params["class_weight"],
    })

    # Metrics
    mlflow.log_metrics({**val_metrics, **test_metrics})

    # Ranking artifact
    rank.to_csv("meta_val_candidate_ranking.csv", index=False)
    mlflow.log_artifact("meta_val_candidate_ranking.csv", artifact_path="model_selection")

    # Signature
    input_example = X_train.head(5)
    sig = infer_signature(input_example, final_meta.predict_proba(input_example)[:, 1])

    mlflow.sklearn.log_model(
        sk_model=final_meta,
        artifact_path="model_final",
        input_example=input_example,
        signature=sig,
    )

    mlflow.set_tag('model_stage', 'frozen_baseline')
    mlflow.set_tag('baseline_name', 'fusion_lr_baseline_tcs_2021_2023_sent_lag_1_gated')
    mlflow.set_tag('selection_protocol', 'train_timeseries_cv_val_select_refit_trainval')
    mlflow.set_tag("expert_type", "fusion")
    mlflow.set_tag("label_col", label_col)
    mlflow.set_tag("sentiment_expert", "lag-1 minimal (6 features)")
    mlflow.set_tag("fusion_result", "same as OHLCV baseline")
    mlflow.set_tag("additional_comments", "Sentiment did not improve ensemble performance under constrained single-symbol data.")


    # Optional registry
    model_name = f"fusion_meta_lr_tcs_{meta_ohlcv.get('start_year','2021')}_{meta_ohlcv.get('end_year','2023')}_h{horizon_days}d_sent_lag_1_gated"
    mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/model_final", model_name)

print("Done. Run:", mlflow.active_run())


Successfully registered model 'fusion_meta_lr_tcs_2021_2023_h5d_sent_lag_1_gated'.
2026/01/26 15:05:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: fusion_meta_lr_tcs_2021_2023_h5d_sent_lag_1_gated, version 1
Created version '1' of model 'fusion_meta_lr_tcs_2021_2023_h5d_sent_lag_1_gated'.


🏃 View run stacked_meta_lr_tcs_2021_2023_h5d_sent_lag_1_gated at: https://dagshub.com/Roncool13/dissertation-mlflow.mlflow/#/experiments/3/runs/fba000bff6a64c5180f40b3317fd51de
🧪 View experiment at: https://dagshub.com/Roncool13/dissertation-mlflow.mlflow/#/experiments/3
Done. Run: None


## 11) Visualizations (ROC-Curve)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

val_scores = final_meta.predict_proba(meta_val)[:, 1]
test_scores = final_meta.predict_proba(meta_test)[:, 1]

fig, ax = plt.subplots(figsize=(6, 4))
RocCurveDisplay.from_predictions(y_meta_val, val_scores, name="Val", ax=ax)
RocCurveDisplay.from_predictions(y_meta_test, test_scores, name="Test", ax=ax)
ax.set_title("Fusion Meta-Model ROC Curves")
plt.show()